### Installing Dependencies

In [2]:
import os, sys

import matplotlib.pyplot as plt 

import numpy as np 
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

from sklearn.linear_model import LinearRegression

In [3]:
train_data = pd.read_csv('blogData_train.csv')

#### Feature Descriptions

In [4]:
# 1...50: 
#       Average, standard deviation, min, max and median of the 
#       Attributes 51...60 for the source of the current blog post
#       With source we mean the blog on which the post appeared. 
#       For example, myblog.blog.org would be the source of 
#       the post myblog.blog.org/post_2010_09_10 
# 51:   Total number of comments before basetime
# 52:   Number of comments in the last 24 hours before the 
#       basetime
# 53:   Let T1 denote the datetime 48 hours before basetime,
#       Let T2 denote the datetime 24 hours before basetime.
#       This attribute is the number of comments in the time period 
#       between T1 and T2
# 54:   Number of comments in the first 24 hours after the 
#       publication of the blog post, but before basetime
# 55:   The difference of Attribute 52 and Attribute 53
# 56...60: 
#       The same features as the attributes 51...55, but  
#       features 56...60 refer to the number of links (trackbacks), 
#       while features 51...55 refer to the number of comments.
# 61:   The length of time between the publication of the blog post 
#       and basetime
# 62:   The length of the blog post
# 63...262: 
#       The 200 bag of words features for 200 frequent words of the 
#       text of the blog post
# 263...269: binary indicator features (0 or 1) for the weekday
#       (Monday...Sunday) of the basetime
# 270...276: binary indicator features (0 or 1) for the weekday
#       (Monday...Sunday) of the date of publication of the blog
#       post
# 277:  Number of parent pages: we consider a blog post P as a
#       parent of blog post B, if B is a reply (trackback) to 
#       blog post P.
# 278...280:  
#       Minimum, maximum, average number of comments that the 
#       parents received
# 281:  The target: the number of comments in the next 24 hours
#       (relative to basetime)

### Feature Normalisation

In [5]:
features_to_normalise = train_data.columns[50:60]; features_to_normalise

Index(['base_comments', 'comments_1d', 'comments_2d', 'comments_firstDay',
       'comments_diff', 'base_links', 'links_1d', 'links_2d', 'links_firstDay',
       'links_diff'],
      dtype='object')

In [6]:
train_data.columns[20:25]

Index(['comments_diff_avg', 'comments_diff_stdev', 'comments_diff_min',
       'comments_diff_max', 'comments_diff_med'],
      dtype='object')

In [7]:
normalised_df = train_data.assign(**{
    feat + '_in_site_stdevs': np.where(
        train_data[feat + '_stdev'] == 0, 0,
        (train_data[feat] - train_data[feat + '_avg']) / train_data[feat + '_stdev']
    )
    for feat in features_to_normalise
})

In [37]:
normalised_df

,base_comments_avg,base_comments_stdev,base_comments_min,base_comments_max,base_comments_med,comments_1d_avg,comments_1d_stdev,comments_1d_min,comments_1d_max,comments_1d_med,...,base_comments_in_site_stdevs,comments_1d_in_site_stdevs,comments_2d_in_site_stdevs,comments_firstDay_in_site_stdevs,comments_diff_in_site_stdevs,base_links_in_site_stdevs,links_1d_in_site_stdevs,links_2d_in_site_stdevs,links_firstDay_in_site_stdevs,links_diff_in_site_stdevs
0,40.30467,53.845657,0,401,15.0,15.52416,32.441880,0,377,3.0,...,-0.711379,-0.416873,-0.430601,-0.671840,0.011260,-0.599397,-0.371480,-0.351476,-0.570769,-0.015075
1,40.30467,53.845657,0,401,15.0,15.52416,32.441880,0,377,3.0,...,-0.637093,-0.416873,-0.307959,-0.609953,-0.075345,-0.599397,-0.371480,-0.351476,-0.570769,-0.015075
2,40.30467,53.845657,0,401,15.0,15.52416,32.441880,0,377,3.0,...,-0.637093,-0.416873,-0.307959,-0.609953,-0.075345,-0.599397,-0.371480,-0.351476,-0.570769,-0.015075
3,40.30467,53.845657,0,401,15.0,15.52416,32.441880,0,377,3.0,...,-0.711379,-0.416873,-0.430601,-0.671840,0.011260,-0.599397,-0.371480,-0.351476,-0.570769,-0.015075
4,40.30467,53.845657,0,401,15.0,15.52416,32.441880,0,377,3.0,...,-0.692807,-0.447698,-0.369280,-0.671840,-0.053693,-0.599397,-0.371480,-0.351476,-0.570769,-0.015075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52392,33.00000,0.000000,33,33,33.0,11.00000,15.556349,0,33,0.0,...,0.000000,-0.707107,1.414214,0.000000,-1.224745,0.000000,0.000000,0.000000,0.000000,0.000000
52393,33.00000,0.000000,33,33,33.0,11.00000,15.556349,0,33,0.0,...,0.000000,-0.707107,-0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
52394,0.00000,0.000000,0,0,0.0,0.00000,0.000000,0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.414214,-0.707107,0.000000,1.224745
52395,0.00000,0.000000,0,0,0.0,0.00000,0.000000,0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.707107,1.414214,0.000000,-1.224745


In [ ]:
normalised_df['base_day'] 

#### Building Model

In [22]:
#normalised features, base weekday, published weekday, normalised parent page
feature_names = ['baseDay_Mon', 'baseDay_Tue', 'baseDay_Wed', 'baseDay_Thu', 'baseDay_Fri', 'baseDay_Sat', 'baseDay_Sun', 'pubDay_Mon', 'pubDay_Tue', 'pubDay_Wed', 'pubDay_Thu', 'pubDay_Fri', 'pubDay_Sat', 'pubDay_Sun', 'base_comments_in_site_stdevs','comments_1d_in_site_stdevs','comments_2d_in_site_stdevs','comments_firstDay_in_site_stdevs','comments_diff_in_site_stdevs','base_links_in_site_stdevs','links_1d_in_site_stdevs','links_2d_in_site_stdevs','links_firstDay_in_site_stdevs','links_diff_in_site_stdevs']
features = normalised_df[feature_names]

#target
labels = normalised_df[['target']]

In [31]:
model = LinearRegression(fit_intercept=False)
model.fit(features, labels)

LinearRegression(fit_intercept=False)

In [33]:
pd.Series(model.coef_[0], index=feature_names).sort_values()

links_diff_in_site_stdevs          -12.510032
links_2d_in_site_stdevs             -8.615898
comments_diff_in_site_stdevs        -8.195621
comments_2d_in_site_stdevs          -5.671013
base_links_in_site_stdevs           -2.840567
base_comments_in_site_stdevs        -2.822316
comments_firstDay_in_site_stdevs    -0.189903
links_firstDay_in_site_stdevs        1.136192
pubDay_Sun                           2.013112
baseDay_Sat                          2.073758
baseDay_Fri                          2.213780
pubDay_Mon                           2.407629
baseDay_Sun                          2.902554
pubDay_Sat                           2.948039
baseDay_Mon                          3.109126
baseDay_Thu                          3.530883
pubDay_Tue                           3.721612
pubDay_Thu                           3.780070
pubDay_Wed                           3.888155
baseDay_Wed                          4.438637
pubDay_Fri                           4.818555
baseDay_Tue                       

In [34]:
normalised_df['pubDay_Sun'].value_counts()

pubDay_Sun
0    47738
1     4659
Name: count, dtype: int64

In [32]:
model.score(features, labels)

0.026429490392339994

In [35]:
model.intercept_

0.0